<a href="https://colab.research.google.com/github/csch7/CSCI-4170/blob/main/Homework-06/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://paperswithcode.com/dataset/xl-sum

In [1]:
!pip install datasets

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from transformers import BartTokenizer, BartForConditionalGeneration
from datasets import load_dataset

from google.colab import drive

dat = load_dataset('csebuetnlp/xlsum', 'english')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
train = dat['train'].to_pandas()
test = pd.concat([dat['test'].to_pandas(), dat['validation'].to_pandas()])

print(train.shape, test.shape)

(306522, 5) (23070, 5)


In [4]:
train = train.iloc[:230700]
print(train.shape, test.shape)

(230700, 5) (23070, 5)


In [5]:
print(train.head())

                                       id  \
0                       uk-wales-56321577   
1  uk-scotland-highlands-islands-11069985   
2               uk-england-leeds-45776523   
3                world-us-canada-51010441   
4       uk-scotland-glasgow-west-52274685   

                                                 url  \
0         https://www.bbc.com/news/uk-wales-56321577   
1  https://www.bbc.com/news/uk-scotland-highlands...   
2  https://www.bbc.com/news/uk-england-leeds-4577...   
3  https://www.bbc.com/news/world-us-canada-51010441   
4  https://www.bbc.com/news/uk-scotland-glasgow-w...   

                                               title  \
0  Weather alert issued for gale force winds in W...   
1   Huge tidal turbine installed at Orkney test site   
2  Leeds stabbing: Man attacked outside betting shop   
3  Could killing of Iranian general help Trump ge...   
4  Coronavirus: 'I've moved out to protect my fam...   

                                             summary  \

In [6]:
train = train.drop(columns=['id', 'url', 'title'])
test = test.drop(columns=['id', 'url', 'title'])


In [7]:
print(train.head(), test.head())

                                             summary  \
0  Winds could reach gale force in Wales with sto...   
1  The massive tidal turbine AK1000 has been inst...   
2  A man has been stabbed in broad daylight outsi...   
3  It was inevitable that the fallout from the US...   
4  Week four of social distancing is starting to ...   

                                                text  
0  The Met Office has issued a yellow weather war...  
1  Atlantis Resources unveiled the marine energy ...  
2  Police were called to the scene outside the Co...  
3  Anthony ZurcherNorth America reporter@awzurche...  
4  By Debbie JacksonBBC Scotland But while most o...                                                summary  \
0  Donald Trump campaigned on becoming a presiden...   
1  The Welsh Government "would probably" take the...   
2  A university has mistakenly emailed hundreds o...   
3  Some progress has been made in encouraging gir...   
4  The president of business organisation, the CB... 

In [8]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [21]:
import torch
from tqdm import tqdm

encoded_articles = []
encoded_summaries = []

for i in tqdm(range(10000), total=10000):
  encoded_articles.append(torch.tensor(tokenizer.encode(train.text.iloc[i][:500], max_length=5)))
  encoded_summaries.append(torch.tensor(tokenizer.encode(train.summary.iloc[i], max_length=5)))


100%|██████████| 10000/10000 [00:24<00:00, 401.11it/s]


In [27]:
from torch import optim
from torch.utils.data import DataLoader

def train_Bart(data, labels, model, batch_size = 32, epochs = 10, lr = 0.001):

  model.train()
  optimizer = optim.AdamW(model.parameters(), lr)

  data = [data] + [labels]
  train_loader = DataLoader(data, batch_size, shuffle=True)

  for e in range(epochs):

    train_loader = iter(train_loader)
    i = 0
    while i < len(train_loader):
      optimizer.zero_grad()
      batch = next(train_loader)
      print(len(batch), batch[1].shape)
      model_output = model(batch[0], labels=batch[1])
      loss = model_output.loss
      print(loss.item())
      loss.backward()
      optimizer.step()
      i+=1


train_Bart(encoded_articles, encoded_summaries, model)

10000 torch.Size([2, 5])
12.803457260131836


KeyboardInterrupt: 